In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import os

from modules.Table import TableUserInferface

In [2]:
tui_actual = TableUserInferface()

In [3]:
path = '/app/export/*'
folders = [os.path.split(folder)[-1] for folder in glob(path)];

In [4]:
for folder in folders:
    tui_actual.load_heatmaps_from_folder(os.path.join('/app/export', folder))

    tui_actual.calc_sidewalk_kpis() # Listo
    tui_actual.calc_bicycle_route_kpis() # Listo

    tui_actual.num_proximity_to_json() # Listo
    tui_actual.num_land_uses_to_json() # Listo
    tui_actual.radar_proximity_to_json() # Listo
    tui_actual.radar_num_land_uses() # Listo
    tui_actual.radar_densities_to_json() # Listo
    tui_actual.bar_densities_to_json() # Listo

    import json
    json_export = {}
    json_export.update(tui_actual.radar_kpis)
    json_export.update(tui_actual.proximity_kpis)
    json_export.update(tui_actual.land_uses_kpis)
    json_export.update(tui_actual.density_kpis)
    radar_list = json_export['radar']['valoresSet1']
    json_export['radar']['valoresSet2'] = [75]*len(radar_list)

    # Imprimir el JSON con formato y sangría
    # Abrir un archivo para escritura
    pattern = os.path.split(folder)[-1]
    json_filename = f"/app/data/jsons/{pattern}.json"
    with open(json_filename, "w") as archivo:
        # Con json.dump() y el argumento indent
        json.dump(json_export, archivo, indent=4)

    tui_actual.reset_json_data()

In [4]:
path = '/app/export'
# folder = os.path.join(path, '0000000')
folder = os.path.join(path, '1111111')
tui_actual.load_heatmaps_from_folder(folder)

tui_actual.calc_sidewalk_kpis() # Listo
tui_actual.calc_bicycle_route_kpis() # Listo

tui_actual.num_proximity_to_json() # Listo
tui_actual.num_land_uses_to_json() # Listo
tui_actual.radar_proximity_to_json() # Listo
tui_actual.radar_num_land_uses() # Listo
tui_actual.radar_densities_to_json() # Listo
tui_actual.bar_densities_to_json() # Listo

import json
json_export = {}
json_export.update(tui_actual.radar_kpis)
json_export.update(tui_actual.proximity_kpis)
json_export.update(tui_actual.land_uses_kpis)
json_export.update(tui_actual.density_kpis)
radar_list = json_export['radar']['valoresSet1']
json_export['radar']['valoresSet2'] = [75]*len(radar_list)
normalizer_multiply_radar = [1,
  1,
  1,
  1,
  1,
  1,
  1,
  100,
  0.01,
  10.0,
  1,
  0.002,
  5]
radar_list_norm = [value*norm for value, norm in zip(radar_list, normalizer_multiply_radar)]
json_export['radar']['valoresSet1'] = radar_list_norm


normalizer_multiply_stackedbar = [
    1,
    1/10,
    1/20,
]

l = json_export['barrasStackeadas']['barras']

for index, dt in enumerate(l):
    for k, v in dt.items():
        if(k=='valores'):
            json_export['barrasStackeadas']['barras'][index]['valores'] = [value*norm for value, norm in zip(v, normalizer_multiply_stackedbar)]

# Imprimir el JSON con formato y sangría
# Abrir un archivo para escritura
pattern = os.path.split(folder)[-1]
json_filename = f"/app/data/jsons/{pattern}.json"
with open(json_filename, "w") as archivo:
    # Con json.dump() y el argumento indent
    json.dump(json_export, archivo, indent=4)

tui_actual.reset_json_data()

In [5]:
json_export['barrasStackeadas']['barras']

[{'nombre': 'Aurora de Chile', 'valores': [10.0, 471.0, 5636.286684794872]},
 {'nombre': 'Pedro de Valdivia', 'valores': [4.0, 2094.0, 11918.126309182951]},
 {'nombre': 'Pedro Del Rio', 'valores': [16.0, 2288.0, 10777.649150760586]}]

[94.85441932093399, 68.9367555317033, 85.09599593001447, 79.61900844963557, 86.36283997940997, 90.9137235227095, 78.43124511902455, 0.6226427655878166, 8019.0, 10.0, 43.0, 33029.81345620307, 15.0]
[94.85441932093399, 68.9367555317033, 85.09599593001447, 79.61900844963557, 86.36283997940997, 90.9137235227095, 78.43124511902455, 62.26427655878166, 80.19, 100.0, 43.0, 66.05962691240613, 75.0]


In [ ]:
df = tui_actual.calc_numeric_densities()

In [ ]:
neighborhoods = tui_actual.neighborhoods
nh = pd.concat(neighborhoods.values())

In [ ]:
df = gpd.sjoin(df, nh)

In [ ]:
df.rename(columns={'Nombre': 'barrio'}, inplace=True)
dens_cols = ['hex_id', 'Category', 'density', 'barrio', 'geometry']
df = df[dens_cols]
df['area'] = df['geometry'].area

In [ ]:
vals_to_replace = ['salud', 'cultura', 'educacion']
for val in vals_to_replace:
    df['Category'] = df['Category'].str.replace(val, 'amenities')

In [ ]:
group_df = df.groupby(['Category', 'barrio'])[['density', 'area']].agg('sum').reset_index()
group_df.rename(columns={'density': 'count'}, inplace=True)
group_df = group_df[['Category','barrio','count']]
# Función para capitalizar palabras con más de 2 caracteres
def capitalizar_palabras(texto):
    palabras = texto.split()
    palabras_capitalizadas = [palabra.capitalize() if len(palabra) > 2 else palabra.lower() for palabra in palabras]
    return " ".join(palabras_capitalizadas)

# Aplicar la función a la columna "barrio"
df["barrio"] = df["barrio"].apply(capitalizar_palabras)

In [ ]:
# Obtener la lista de categorías únicas
categorias = df["Category"].unique()

# Crear un diccionario para almacenar los datos
resultados = {}

for categoria in categorias:
    df_categoria = group_df[group_df["Category"] == categoria]
    barras = []

    for index, row in df_categoria.iterrows():
        nombre = row["barrio"]
        valores = group_df[group_df["barrio"] == nombre]["count"].tolist()
        
        barra = {"nombre": nombre, "valores": valores}
        barras.append(barra)

    resultados[categoria] = {"barras": barras}

resultados

In [ ]:
mask = df['Category'].isin(['salud', 'cultura', 'educacion'])
df.loc[mask, 'Category'] = 'amenities'

In [ ]:
group_df = df.groupby(['hex_id', 'Category', 'geometry'])['density'].agg('sum').reset_index()

In [ ]:
group_df = group_df[group_df['density'] > 0]

In [ ]:
group_df = gpd.GeoDataFrame(data=group_df.drop(columns=['geometry']), geometry=group_df['geometry'])

In [ ]:
group_df['area'] = group_df['geometry'].area

In [ ]:
group_df

In [ ]:
df[df['Category']=='population_density']

In [ ]:
df = tui_actual.heat_maps['building_density'].copy()
df.loc[df['density']>0, 'density'].mean()
# df.loc[df['density']>=0, 'density'].mean()

In [ ]:
df

In [ ]:
def mapeo_inverso_tiempo_viaje(valor):
    limit_time = 30

    if valor >= limit_time:
        return 0
    else:
        return 100 - (valor / limit_time) * 100

mapeo_inverso_tiempo_viaje(15)

In [ ]:
df = tui_actual.calc_radar_proximity()

delete_categories = ['Aprovisionamiento']
df['Category'] = df['Category'].str.replace('Cuidados', 'Salud')
df = df[~df['Category'].isin(delete_categories)]

df

In [ ]:
ppl = tui_actual.heat_maps['population_density']
ppl = ppl[['hex_id', 'population']]
total_population = ppl['population'].sum()

In [ ]:
proximity_gdf = []

proximity_cols = ['source', 'destination', 'path_lengths', 'Category', 'hex_id', 'geometry', 'travel_time']

listado_de_claves = list(tui_actual.heat_maps.keys())

claves_proximidad = [clave for clave in listado_de_claves if clave.endswith('_proximity') and 'parque' not in clave and 'plaza' not in clave]

for key in claves_proximidad:
    gdf = tui_actual.heat_maps[key].drop_duplicates(subset=['hex_id'])
    gdf = gdf[proximity_cols]
    proximity_gdf.append(gdf)

claves_proximidad = [clave for clave in listado_de_claves if clave.endswith('_proximity') and (clave.startswith('plaza') or clave.startswith('parque'))]

for key in claves_proximidad:
    gdf = tui_actual.heat_maps[key].drop_duplicates(subset=['hex_id'])
    gdf['Category'] = gdf['class'].map({'PK': 'Parque', 'SQ': 'Plaza'})
    gdf = gdf[proximity_cols]
    # print(key)
    proximity_gdf.append(gdf)

proximity_gdf = pd.concat(proximity_gdf)
proximity_gdf.head()

In [ ]:
# Definimos los rangos
rangos = [0, 10, 20]

# Etiquetas para los rangos, ajusta según tus necesidades
etiquetas = ['Baja', 'Media', 'Alta']

# Creamos una nueva columna en el gdf para almacenar las etiquetas de los rangos
proximity_gdf['proximidad'] = pd.cut(proximity_gdf['travel_time'], bins=rangos + [float('inf')], labels=etiquetas, right=False)

proximity_and_ppl_gdf = pd.merge(proximity_gdf, ppl, on='hex_id', how='outer')

df = proximity_and_ppl_gdf.groupby(['Category', 'proximidad'])['population'].agg('sum').reset_index()

df['population'] = 100*df['population']/total_population

In [ ]:
# Crear un diccionario en el formato deseado
output = {
    "barrasHorizontalesStackeadas": {
        "barras": []
    }
}

# Iterar sobre las filas del DataFrame para crear la estructura deseada
for nombre, group in df.groupby("Category"):
    valores = group["population"].tolist()
    output["barrasHorizontalesStackeadas"]["barras"].append({"nombre": nombre, "valores": valores})

# Convertir a JSON
import json

json_output = json.dumps(output, indent=2)
print(json_output)

In [ ]:
# Preparar los datos para el gráfico
categorias = df['Category'].unique()
proximidades = df['proximidad'].unique()

# Base para las barras apiladas
bases = {categoria: 0 for categoria in categorias}

# Crear el gráfico
fig, ax = plt.subplots()

for proximidad in proximidades:
    valores = df[df['proximidad'] == proximidad].groupby('Category')['population'].sum()
    ax.barh(categorias, valores, left=[bases[categoria] for categoria in categorias], label=proximidad)
    # Actualizar las bases para la siguiente proximidad
    for categoria in categorias:
        bases[categoria] += valores.get(categoria, 0)

# Añadir detalles al gráfico
ax.set_xlabel('Población')
ax.set_title('Distribución de Población por Categoría y Proximidad')
plt.legend()

# Mostrar el gráfico
plt.show()

In [ ]:
land_uses_diversity = tui_actual.heat_maps['land_uses_diversity']
land_uses_diversity.loc[land_uses_diversity['diversity']>0, 'diversity'].mean()

In [ ]:
lu = tui_actual.heat_maps['land_uses_diversity']

In [ ]:
gdf = tui_actual.lu.get_current_land_uses()

In [ ]:
porcentaje_limit = 3

# Calcular la distribución de Uso según area_predio
uso_distribucion = gdf.groupby('Uso')['area_predio'].sum().reset_index()

# Cambiar el nombre de la columna 'area_predio' a 'total_area_predio'
uso_distribucion = uso_distribucion.rename(columns={'area_predio': 'total_area_predio'})

# Calcular el porcentaje de área en relación con el total
uso_distribucion['porcentaje_area'] = uso_distribucion['total_area_predio'] / uso_distribucion['total_area_predio'].sum() * 100

uso_distribucion.loc[uso_distribucion['porcentaje_area']<porcentaje_limit,'Uso'] = 'OTROS'
uso_distribucion = uso_distribucion.groupby(['Uso'])[['total_area_predio', 'porcentaje_area']].agg('sum').reset_index()
# Encuentra la longitud máxima en la columna "Uso"
longitud_maxima = uso_distribucion['Uso'].str.len().max()

# Rellena los valores más cortos con espacios en blanco para que tengan la misma longitud
uso_distribucion['Uso'] = uso_distribucion['Uso'].str.capitalize()
uso_distribucion['Uso'] = uso_distribucion['Uso'].str.strip()
uso_distribucion['Uso'] = uso_distribucion['Uso'].str.ljust(longitud_maxima)

# Crear una lista de diccionarios en el formato deseado
tipos = []
for index, row in uso_distribucion.iterrows():
    tipo = {
        "nombre": row['Uso'],  # Elimina espacios en blanco al principio y al final
        "valor": int(row['porcentaje_area'])  # Convierte el porcentaje a entero
    }
    tipos.append(tipo)

# Crear el diccionario JSON final
json_output = {
    "graficoDeTorta": {
        "tipos": tipos
    }
}

# Convertir a JSON y mostrarlo o guardar en un archivo
print(json.dumps(json_output, indent=2))


In [ ]:
import matplotlib.pyplot as plt

# Crear el gráfico de torta
plt.figure(figsize=(8, 8))
plt.pie(uso_distribucion['total_area_predio'], labels=uso_distribucion['Uso'], autopct='%1.1f%%', startangle=140)
plt.axis('equal')  # Hace que el gráfico de torta sea un círculo

# Añadir un título
plt.title("Distribución de Área por Uso")

# Mostrar el gráfico
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd

# Asumiendo que ya tienes tu GeoDataFrame 'gdf' y la columna 'proximidad' definida

# Contar la cantidad de registros en cada categoría
conteo = gdf['proximidad'].value_counts()

# Calcular los porcentajes
porcentajes = conteo / conteo.sum() * 100

# Preparar los datos para el gráfico de barras apiladas
categorias = porcentajes.index
valores = porcentajes.values

# Iniciar la base para la barra apilada
base = 0

# Crear una barra apilada para cada categoría
for i, valor in enumerate(valores):
    plt.bar("Total", valor, bottom=base, label=categorias[i])
    base += valor

# Añadir detalles al gráfico
plt.ylabel('Porcentaje (%)')
plt.title('Distribución de Proximidad (Normalizado)')
plt.legend()

# Mostrar el gráfico
plt.show()


In [ ]:
def proximity_to_numeric_categories(gdf):
    data = []


    return data

proximity_to_numeric_categories()

In [ ]:
# folder = 
# tui_actual

In [ ]:
gdf = tui_actual.heat_maps['land_uses_diversity']

In [ ]:
gdf.to_file('/app/data/export_as_shape/land_uses/actual')

In [ ]:
def get_changes_data(tui, mode, plate_id, scenario_id):
    output_data = {}
    if mode=='am':
        output_data['actual'] = tui.am.get_current_amenities()
        tui.am._update_plate_area(plate_id, scenario_id)
        output_data['update'] = tui.am.get_current_amenities()
    elif mode=='bk':
        output_data['actual'] = tui.bk.get_current_scenario()
        tui.bk._update_plate_area(plate_id, scenario_id)
        output_data['update'] = tui.bk.get_current_scenario()
    elif mode=='bl':
        output_data['actual'] = tui.bl.get_current_scenario()
        tui.bl._update_plate_area(plate_id, scenario_id)
        output_data['update'] = tui.bl.get_current_scenario()
    elif mode=='ga':
        output_data['actual'] = tui.ga.get_green_areas()
        tui.ga._update_plate_area(plate_id, scenario_id)
        output_data['update'] = tui.ga.get_green_areas()
    elif mode=='lu':
        output_data['actual'] = tui.lu.get_current_land_uses()
        tui.lu._update_plate_area(plate_id, scenario_id)
        output_data['update'] = tui.lu.get_current_land_uses()
    return output_data

mode = 'lu'
sel_plate = 1
scenario_id = 1

changed_data = get_changes_data(tui_actual, mode, sel_plate, scenario_id)

# tui_actual.am.get_current_amenities()
fig, ax = plt.subplots(1,2)
for idx, plate in tui_actual.plates.items():
    if idx!=sel_plate:
        plate.plot(ax=ax[0], alpha=0.5)
    else:
        plate.plot(ax=ax[0], alpha=0.5, color='red')
data = changed_data['actual']
gpd.sjoin(data, tui_actual.area_scope).plot(ax=ax[0], column='plate_id', markersize=10)
print(data.shape)

for idx, plate in tui_actual.plates.items():
    if idx!=sel_plate:
        plate.plot(ax=ax[1], alpha=0.5)
    else:
        plate.plot(ax=ax[1], alpha=0.5, color='red')

data = changed_data['update']
gpd.sjoin(data, tui_actual.area_scope).plot(ax=ax[1], column='plate_id', markersize=10)
print(data.shape)

In [ ]:
plate_id = 7
scenario_id = 1
tui_actual.update_plate_status(plate_id=plate_id, scenario_id=scenario_id)

In [ ]:
plate_id = 5
scenario_id = 1
tui_actual.update_plate_status(plate_id=plate_id, scenario_id=scenario_id)

In [ ]:
tui_actual.st.plate_states

In [ ]:
tui_actual.update_plate_status(7, 0)

In [ ]:
nodes, edges = tui_actual.st.get_current_nodes_and_edges()

In [ ]:
gpd.sjoin(edges, tui_actual.area_scope).plot()

In [ ]:
area = tui_actual.area_scope

In [ ]:
anodes = gpd.read_file('/app/data/calles_etiquetadas/streets/streets/actual/nodes')
aedges = gpd.read_file('/app/data/calles_etiquetadas/streets/streets/actual/edges')
fnodes = gpd.read_file('/app/data/calles_etiquetadas/streets/streets/future/nodes')
fedges = gpd.read_file('/app/data/calles_etiquetadas/streets/streets/future/edges')
anodes['x'] = anodes['geometry'].x
anodes['y'] = anodes['geometry'].y
fnodes['x'] = fnodes['geometry'].x
fnodes['y'] = fnodes['geometry'].y
aedges['from'] = aedges['u']
aedges['to'] = aedges['v']
edges_cols = ['u', 'v', 'key', 'from', 'to', 'osmid', 'length', 'plate_id', 'geometry']
aedges = aedges[edges_cols]
fedges = fedges[edges_cols]
fedges['plate_id'].fillna(0, inplace=True)

In [ ]:
gpd.sjoin(fedges, area).plot(column='plate_id', markersize=5)

In [ ]:
import geopandas as gpd
import glob
import os
def load_plates():
    default_crs = 32718
    plates_path = '/app/assets/plates'
    plates_files = glob.glob(os.path.join(plates_path, '*'))
    plates = {}
    num_plates = 0
    for file in plates_files:
        idx = os.path.split(file)[-1]
        if not idx.isdigit():
        # Verifica si el nombre del archivo es un plate con numero
            continue
        else:
        # Convierte idx en un entero antes de agregarlo al diccionario
            idx = int(idx)
            plate = gpd.read_file(file).to_crs(default_crs)
            plates[idx] = plate
            num_plates += 1
    return num_plates, plates

In [ ]:
num_plates, plates = load_plates()

In [ ]:
from modules.Base import BaseModule

mod = BaseModule()

In [ ]:
mod.plate_states[2]=1
mod.plate_states[6]=1

In [ ]:
plate_state = mod.plate_states

def valid_plates_street_state():
    lower_plate = 6
    for key, value in plate_state.items():
        if key >= lower_plate and value == 1:
            return 1
    return 0

valid_plates_street_state()

In [ ]:
import geopandas as gpd

In [ ]:
gpd.read_parquet('/app/export/0000011/aprovisionamiento_proximity.parquet').plot(column='travel_time')

In [ ]:
fedges.plot(column='plate_id', markersize=5)

In [ ]:
keyword = 'proximity'
filtered_keys = [clave for clave in tui_actual.heat_maps.keys() if keyword in clave]
filtered_keys

In [ ]:
heatmaps = tui_actual.heat_maps

In [ ]:
################################################################
# Hacer el join entre tiempos de viaje y personas por hexagono.
# Sacar cantidad de personas a X minutos de distancia

In [ ]:
gdf = heatmaps['parque_proximity']
gdf.head(3)

In [ ]:
green_areas = tui_actual.ga.get_green_areas()
green_areas.head(3)

In [ ]:
ppl = tui_actual.population_by_unit
ppl.head(3)

In [ ]:
merge_gdf = pd.merge(gdf[['hex_id', 'ID_AV']], ppl[['hex_id', 'population']], on='hex_id')

In [ ]:
group_df = merge_gdf.groupby(['ID_AV'])['population'].agg('sum').reset_index()

In [ ]:
pd.merge(green_areas, group_df, on='ID_AV').plot(column='population', cmap='YlGn')

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
green_areas.plot(ax=ax, color='green')
green_areas[green_areas['ID_AV']=='PK20'].plot(ax=ax, color='red')

In [ ]:
tui_futuro = TableUserInferface()
tui_futuro.change_scenario(1)
tui_futuro.calc_heatmaps_kpis()

In [ ]:
fig, ax = plt.subplots(1,2)
tui_actual.heat_maps['building_density'].plot(column='density', ax=ax[0])
tui_futuro.heat_maps['building_density'].plot(column='density', ax=ax[1])

In [ ]:
fig, ax = plt.subplots(1,2)
tui_actual.heat_maps['land_uses_diversity'].plot(column='diversity', ax=ax[0])
tui_futuro.heat_maps['land_uses_diversity'].plot(column='diversity', ax=ax[1])

In [ ]:
fig, ax = plt.subplots(1,2)
tui_actual.heat_maps['population_density'].plot(column='density', ax=ax[0])
tui_futuro.heat_maps['population_density'].plot(column='density', ax=ax[1])

In [ ]:
lu_actual = tui_actual.heat_maps['land_uses_diversity']
lu_futuro = tui_futuro.heat_maps['land_uses_diversity']
print('Diversidad de Usos de Suelo promedio actual')
print(lu_actual.loc[lu_actual['diversity'] > 0, 'diversity'].mean())
print()
print('Diversidad de Usos de Suelo promedio futuro')
print(lu_futuro.loc[lu_futuro['diversity'] > 0, 'diversity'].mean())

In [ ]:
bd_actual = tui_actual.heat_maps['building_density']
bd_futuro = tui_futuro.heat_maps['building_density']

print('Densidad de construcciones promedio actual')
print(bd_actual.loc[bd_actual['density'] > 0, 'density'].mean())
print()
print('Densidad de construcciones promedio futuro')
print(bd_futuro.loc[bd_futuro['density'] > 0, 'density'].mean())

In [ ]:
pd_actual = tui_actual.heat_maps['population_density']
pd_futuro = tui_futuro.heat_maps['population_density']

print('Densidad de población promedio actual')
print(pd_actual.loc[pd_actual['density'] > 0, 'density'].mean())
print()
print('Densidad de población promedio futuro')
print(pd_futuro.loc[pd_futuro['density'] > 0, 'density'].mean())

In [ ]:
tui_actual.save_heatmaps()
tui_futuro.save_heatmaps()